In [105]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Cargar datos

In [106]:
construido = pd.read_parquet("construido.parquet")
educacion = pd.read_parquet("education2.parquet")
income = pd.read_parquet("income.parquet")
over65 = pd.read_parquet("over65.parquet")
poblacion = pd.read_parquet("poblacion.parquet")
renta = pd.read_parquet("renta.parquet")
turismo = pd.read_parquet("tourism.parquet")
waste = pd.read_parquet("waste.parquet")

## Generar columnas de íncide (País ISO, nombre país y año)

In [107]:
yea = list(waste.YEA.unique())
yea.sort()
print(len(yea))

cou_rem = ['OECD - Total', 'OECD - Europe', 'OECD Asia Oceania', 'OECD America']
cou = waste.Country.unique()
countr = [a for a in cou if a not in cou_rem]
print(len(countr))

cou_rem = ['OECD', 'OECDE', 'OECDAO', 'OECDAM']
cou = waste.COU.unique()
COUNTR = [a for a in cou if a not in cou_rem]
print(len(COUNTR))

years = yea*len(countr)
print(len(years))

countries = [item for item in countr for i in range(len(yea))]
print(len(countries))

COUNTRIES = [item for item in COUNTR for i in range(len(yea))]
print(len(COUNTRIES))

28
43
43
1204
1204
1204


In [108]:
df = pd.DataFrame({'Country':countries, 'Year':years, 'COU':COUNTRIES})
df.head()

,Country,Year,COU
0,Australia,1990,AUS
1,Australia,1991,AUS
2,Australia,1992,AUS
3,Australia,1993,AUS
4,Australia,1994,AUS


## Terreno construido
Cuidado, toma el valor anterior conocido, no interpola ni nada.

In [109]:
df["Built"] = [0]*len(COUNTRIES)
construido = construido[construido.MEAS == 'PCNT']
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        df.loc[(df.COU == a) & (df.Year == b), "Built"] = \
            float(construido[(construido.COU == a) & (construido.Year <= b)].tail(1).Value)
df.head()

,Country,Year,COU,Built
0,Australia,1990,AUS,0.12103
1,Australia,1991,AUS,0.12103
2,Australia,1992,AUS,0.12103
3,Australia,1993,AUS,0.12103
4,Australia,1994,AUS,0.12103


## Personas sin la secundaria
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer el dato, el default es 100.

In [110]:
df["Below Secundary"] = [100]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if educacion[(educacion.LOCATION == a) & (educacion.TIME <= b)].empty :
            c = 100
        else:
            c = float(educacion[(educacion.LOCATION == a) & (educacion.TIME <= b)].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Below Secundary"] = c
df.head()

,Country,Year,COU,Built,Below Secundary
0,Australia,1990,AUS,0.12103,100.000000
1,Australia,1991,AUS,0.12103,44.127056
2,Australia,1992,AUS,0.12103,44.127056
3,Australia,1993,AUS,0.12103,47.159046
4,Australia,1994,AUS,0.12103,49.802025


## Mediana de ingresos
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer el dato, el default es 0.

In [119]:
df["Income Median"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if income[(income.LOCATION == a) & (income.TIME <= b) & (income.MEASURE == "MEDIANC")].empty :
            c = 0
        else:
            c = float(income[(income.LOCATION == a) & (income.Year <= b) & (income.MEASURE == "MEDIANC")].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Income Median"] = c
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median
0,Australia,1990,AUS,0.12103,100.000000,0.0
1,Australia,1991,AUS,0.12103,44.127056,0.0
2,Australia,1992,AUS,0.12103,44.127056,0.0
3,Australia,1993,AUS,0.12103,47.159046,0.0
4,Australia,1994,AUS,0.12103,49.802025,0.0
